### Testing Playground for P0 Programs &ndash; Array Concatenation

In [ ]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString
from ST import printSymTab

def runwasm(wasmfile):
    from IPython.core.display import display, Javascript
    display(Javascript("""
    const params = { 
        P0lib: { 
            write: i => this.append_stream({text: '' + i, name: 'stdout'}),
            writeln: () => this.append_stream({text: '\\n', name: 'stdout'}),
            read: () => window.prompt()
        }
    }

    fetch('""" + wasmfile + """') // asynchronously fetch file, return Response object
      .then(response => response.arrayBuffer()) // read the response to completion and stores it in an ArrayBuffer
      .then(code => WebAssembly.compile(code)) // compile (sharable) code.wasm
      .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
    // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
    """))


Write, compile and execute P0 programs for testing using the following steps:
1. Define a string constant containing a P0 program to be validated in the cell below.
2. Update the `compileString` parameter to accept the string constant in the cell below.
3. Select `Kernel > Restart & Run All` to view and execute the target WebAssembly program.

In [ ]:
# Concatenating array with single element array literal
test1 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [4, 8, 12]
    a := a + [1]
    write(a[4])
"""
compileString(test1, "test1.wat")
!wat2wasm --enable-bulk-memory test1.wat || rm test1.wasm
runwasm("test1.wasm")

In [ ]:
# Concatenating array with multiple element array literal
test2 = """
program p
    var a: [1..3] → integer
    var b: [1..3] → integer
    a := [3, 7, 11]
    b := [4, 8, 12]
    a := a + [1, 5, 9]
    write(a[4])
    write(a[5])
    write(a[6])
"""
compileString(test2, "test2.wat")
!wat2wasm --enable-bulk-memory test2.wat || rm test2.wasm
runwasm("test2.wasm")

In [ ]:
# Concatenating array literal with array literal
test3 = """
program p
    var a: [1..3] → integer
    a := [3, 7, 11] + [4, 8, 12]
    write(a[4])
"""
compileString(test3, "test3.wat")
!wat2wasm --enable-bulk-memory test3.wat || rm test3.wasm
runwasm("test3.wasm")

In [ ]:
#Update this line to see WebAssembly of a specific test case
!cat -n "test3.wat"